

"""
DESCRIPTION:
    This sample demonstrates how to use basic agent operations from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_basics.py

    Before running the sample:

    pip install azure-ai-projects azure-identity

    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.
"""


In [1]:
%pip install azure-ai-projects

Note: you may need to restart the kernel to use updated packages.


In [2]:
#https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support?tabs=python#azure-openai-models
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageTextContent

# [START create_project_client]
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
# [END create_project_client]

with project_client:

    # [START create_agent]
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="a1",
        instructions="You are helpful assistant",
    )
    # [END create_agent]
    print(f"Created agent, agent ID: {agent.id}")

    # [START create_thread]
    thread = project_client.agents.create_thread()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

    # [START create_message]
    message = project_client.agents.create_message(thread_id=thread.id, role="user", content="Hello, tell me a joke")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

    # [START create_run]
    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # [START list_messages]
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # The messages are following in the reverse order,
    # we will iterate them and output only text contents.
    for data_point in reversed(messages.data):
        last_message_content = data_point.content[-1]
        if isinstance(last_message_content, MessageTextContent):
            print(f"{data_point.role}: {last_message_content.text.value}")

    # [END list_messages]
    print(f"Messages: {messages}")

Created agent, agent ID: asst_6PSZdiQE3NjxnSuD7vgkBk5z
Created thread, thread ID: thread_hCniEpGMjbkXUOF9t8mrNiCs
Created message, message ID: msg_nfIATD9n6cKRLe02eTBhDHrK
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: Hello, tell me a joke
MessageRole.AGENT: Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!
Messages: {'object': 'list', 'data': [{'id': 'msg_Hsz9F3v4iKcSHsYFlowzasre', 'object': 'thread.message', 'created_at': 1734982441, 'assistant_id': 'asst_6PSZdiQE3NjxnSuD7vgkBk5z', 'thread_id': 'thread_hCniEpGMjbkXUOF9t8mrNiCs', 'run_id': 'run_vUbjHRjQevPxkk0Gapvn5jG1', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_nfIATD9n6cKRLe02eTBhDHrK', 'object': 'thread.message', 'created_

In [ ]:
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# [START list_messages]
messages = project_client.agents.list_messages(thread_id=thread.id)

# The messages are following in the reverse order,
# we will iterate them and output only text contents.
for data_point in reversed(messages.data):
    last_message_content = data_point.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{data_point.role}: {last_message_content.text.value}")

# [END list_messages]
print(f"Messages: {messages}")